In [259]:
## 1. 모듈 로딩
import pandas as pd
import re
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 요청하신 모듈들
from torchmetrics import classification as clf
from torchmetrics.classification import BinaryF1Score, BinaryAccuracy
from konlpy.tag import Okt 

In [260]:
## 2. 데이터 로드 및 정제 (제공해주신 코드 + 정제 로직 추가)
lines = []
files = [
    "./DATA/사람-사람 대화.txt",
    "./DATA/사람-사람 대화2.txt"  
]

# 사람 데이터 읽기
for file in files:
    try:
        with open(file, encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line: continue
                # 맨 앞 번호 및 특수문자 제거
                line = re.sub(r"^\d+[\.\:\)]?\s*", "", line)
                if line:
                    lines.append(line)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없음: {file}")

human_df = pd.DataFrame({"text": lines, "label": 0})
print(human_df.info())

<class 'pandas.DataFrame'>
RangeIndex: 7623 entries, 0 to 7622
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    7623 non-null   str  
 1   label   7623 non-null   int64
dtypes: int64(1), str(1)
memory usage: 119.2 KB
None


In [ ]:
import json

json_files = [
    "./DATA/output_daily_1st.json",
    "./DATA/output_daily_2nd.json",
]

json_lines = []

for jf in json_files:
    try:
        with open(jf, encoding="utf-8") as f:
            data = json.load(f)   # list of dict
            
            for item in data:
                utter = item.get("user_utterance")
                
                # null, None, 빈 문자열 제거
                if utter and utter != "null":
                    utter = utter.strip()
                    if utter:
                        json_lines.append(utter)

    except FileNotFoundError:
        print(f"파일 없음: {jf}")
    
json_df = pd.DataFrame({
    "text": json_lines,
    "label": 0   # 사람 데이터
})

# 기존 txt 기반 human_df와 병합
human_df = pd.concat([human_df, json_df], ignore_index=True)

print(human_df.info())



파일 없음: ./DATA/output_daily_1st.json
<class 'pandas.DataFrame'>
RangeIndex: 13341 entries, 0 to 13340
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    13341 non-null  str  
 1   label   13341 non-null  int64
dtypes: int64(1), str(1)
memory usage: 208.6 KB
None


In [262]:
# AI 데이터 읽기
try:
    ai_df = pd.read_csv("./DATA/사람-AI 대화.csv")
    # 컬럼명이 '대화문'이 아니라면 수정 필요 (파일 확인 결과 'text'일 수 있음)
    if '대화문' in ai_df.columns:
        ai_df = ai_df.rename(columns={"대화문": "text"})
    
    # [중요] AI 답변에서 질문 태그(<s>[INST]...[/INST]) 제거 함수
    def clean_ai_text(text):
        if pd.isna(text): return ""
        text = str(text)
        if "[/INST]" in text:
            text = text.split("[/INST]")[-1] # 태그 뒷부분(답변)만 사용
        text = text.replace("</s>", "").replace("<s>", "").strip()
        text = re.sub(r"^['\"]|['\"]$", "", text) # 앞뒤 따옴표 제거
        return text

    ai_df['text'] = ai_df['text'].apply(clean_ai_text)
    ai_df["label"] = 1
except Exception as e:
    print(f"AI 데이터 로드 중 오류: {e}")
    ai_df = pd.DataFrame(columns=['text', 'label'])

print(ai_df.info())

<class 'pandas.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   text    12000 non-null  str  
 1   label   12000 non-null  int64
dtypes: int64(1), str(1)
memory usage: 187.6 KB
None


In [264]:
# 데이터 병합
df = pd.concat([human_df, ai_df], ignore_index=True)
df = df.dropna(subset=['text']) # 결측치 제거
df = df[df['text'].str.strip() != ""] # 빈 문자열 제거
print(f"전체 데이터 개수: {len(df)}")


전체 데이터 개수: 25341


In [265]:
df["label"].value_counts()

label
0    13341
1    12000
Name: count, dtype: int64

In [266]:
okt = Okt()

# 텍스트 -> 형태소 리스트 변환
X_data = []
for sentence in df['text']:
    # 속도를 위해 stem=True (어간 추출) 사용
    tokenized = okt.morphs(sentence, stem=True) 
    X_data.append(tokenized)

# 단어 사전(Vocabulary) 만들기
word_to_index = {"<PAD>": 0, "<UNK>": 1} # 0: 패딩, 1: 모르는 단어
for sent in X_data:
    for word in sent:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

vocab_size = len(word_to_index)
print(f"단어 사전 크기: {vocab_size}")


단어 사전 크기: 27024


In [267]:
## 4. 텍스트 인코딩 및 패딩
def text_to_sequence(tokenized_sentences, vocab, max_len=50):
    sequences = []
    for sent in tokenized_sentences:
        # 단어를 인덱스로 변환 (없으면 1: <UNK>)
        seq = [vocab.get(word, 1) for word in sent]
        
        # 패딩 (길이 맞추기)
        if len(seq) < max_len:
            seq = seq + [0] * (max_len - len(seq)) # 뒤에 0 채우기
        else:
            seq = seq[:max_len] # 자르기
        sequences.append(seq)
    return torch.tensor(sequences, dtype=torch.long)

MAX_LEN = 50  # 문장 최대 길이 설정
X_tensor = text_to_sequence(X_data, word_to_index, MAX_LEN)
y_tensor = torch.tensor(df['label'].values, dtype=torch.float32).unsqueeze(1)

In [268]:
# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y_tensor
)

In [269]:
# DataLoader 생성
train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)

In [270]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout=0.3):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.dropout = nn.Dropout(dropout)

        self.out = nn.Linear(16, 1)

    def forward(self, x):
        # x: [B, T]
        x = self.embedding(x)              # [B, T, E]

        _, (hidden, _) = self.lstm(x)
        x = hidden[-1]                     # [B, H]

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))

        return torch.sigmoid(self.out(x))

In [271]:
# 하이퍼파라미터
epochs = 30
LR = 0.001
device = "cuda" if torch.cuda.is_available() else "cpu"

ALL_MODEL     = './Models/text_model.pt'   
WEIGHTS_MODEL = './Models/text_weights' 



model = TextClassifier(vocab_size, 100, 64).to(device)
loss_fn = nn.BCELoss() # 이진 분류 손실함수
optimizer = optim.Adam(model.parameters(), lr=LR)




In [272]:
# TorchMetrics 평가 지표 정의
metric_acc = BinaryAccuracy().to(device)
metric_f1 = BinaryF1Score().to(device)

best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    train_loss = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() 
# 검증 (Validation)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            
            # 평가 지표 업데이트
            metric_acc.update(outputs, labels)
            metric_f1.update(outputs, labels)
    
    # 에포크 결과 계산
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    val_acc = metric_acc.compute()
    val_f1 = metric_f1.compute()
    
    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")
   
    # Validation Loss가 개선되었을 때만 모델 저장 (Best Model)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), WEIGHTS_MODEL)
        print(f"  --> Best Model Saved! (Loss: {best_val_loss:.4f})")
    
    # 지표 초기화
    metric_acc.reset()
    metric_f1.reset()

print("학습 완료!")
# 전체 모델 저장 (마지막 상태)
torch.save(model, ALL_MODEL)

Epoch [1/30] Train Loss: 0.2508 | Val Loss: 0.0980 | Val Acc: 0.9629 | Val F1: 0.9595
  --> Best Model Saved! (Loss: 0.0980)
Epoch [2/30] Train Loss: 0.0718 | Val Loss: 0.0778 | Val Acc: 0.9751 | Val F1: 0.9735
  --> Best Model Saved! (Loss: 0.0778)
Epoch [3/30] Train Loss: 0.0489 | Val Loss: 0.0637 | Val Acc: 0.9799 | Val F1: 0.9786
  --> Best Model Saved! (Loss: 0.0637)
Epoch [4/30] Train Loss: 0.0305 | Val Loss: 0.0856 | Val Acc: 0.9803 | Val F1: 0.9791
Epoch [5/30] Train Loss: 0.0253 | Val Loss: 0.0816 | Val Acc: 0.9781 | Val F1: 0.9766
Epoch [6/30] Train Loss: 0.0192 | Val Loss: 0.0796 | Val Acc: 0.9815 | Val F1: 0.9803
Epoch [7/30] Train Loss: 0.0161 | Val Loss: 0.0803 | Val Acc: 0.9805 | Val F1: 0.9793
Epoch [8/30] Train Loss: 0.0255 | Val Loss: 0.0880 | Val Acc: 0.9803 | Val F1: 0.9791
Epoch [9/30] Train Loss: 0.0291 | Val Loss: 0.0986 | Val Acc: 0.9749 | Val F1: 0.9738
Epoch [10/30] Train Loss: 0.0332 | Val Loss: 0.1094 | Val Acc: 0.9730 | Val F1: 0.9714
Epoch [11/30] Train Lo

In [ ]:
## 7. 예측 테스트 함수
model.load_state_dict(torch.load(WEIGHTS_MODEL))
def predict_sentence(text):
    model.eval()
    # 전처리
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text) # 특수문자 제거 등
    tokenized = okt.morphs(text, stem=True)
    seq = [word_to_index.get(w, 1) for w in tokenized]
    
    # 패딩
    if len(seq) < MAX_LEN:
        seq = seq + [0] * (MAX_LEN - len(seq))
    else:
        seq = seq[:MAX_LEN]
        
    input_tensor = torch.tensor([seq], dtype=torch.long).to(device)
    
    with torch.no_grad():
        score = model(input_tensor).item()
    
    print(f"입력: '{text}'")
    if score > 0.5:
        print(f"결과: AI (확률 {score:.4f})")
    else:
        print(f"결과: 사람 (확률 {1-score:.4f})")
    print("-" * 30)

# 테스트
predict_sentence("전에 자료 남아있어서 다시 안해도 되겠다")
predict_sentence("K-디지털 트레이닝(KDT)**은 고용노동부 주관의 국비 지원 디지털 직무 교육이야.AI·웹 개발·데이터 분석 같은 분야에서 실무 중심으로 배울 수 있어.")

입력: '전에 자료 남아있어서 다시 안해도 되겠다'
결과: 사람 (확률 0.9970)
------------------------------
입력: 'K디지털 트레이닝KDT은 고용노동부 주관의 국비 지원 디지털 직무 교육이야AI웹 개발데이터 분석 같은 분야에서 실무 중심으로 배울 수 있어'
결과: AI (확률 0.9981)
------------------------------


C:\Users\keyin\AppData\Local\Temp\ipykernel_3368\1984018874.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(WEIGHTS_MODEL))
